In [29]:
# Trading Bot Steps
# 1. Import necessary libraries
# 2. Set up API credentials/Fetching live data
# 3. Define trading strategy
# 4. Simulate trades
# 5. Integrate different components
# 6. Run the bot
from datetime import datetime
import yfinance as yf

In [ ]:


class MyTradingStrategy:
    def __init__(self,name):
        self.__name = name

    def generate_signals(self, price_data):
        print("This method will be overridden by subclasses")
        return "Hold"
    @property
    def name(self):
        return self.__name

myBaseObject = MyTradingStrategy("Base Strategy")
# print(myBaseObject.name())
print(myBaseObject.name) # due to @property decorator
print(myBaseObject.generate_signals([1,2,3,4,5]))

class MySMATradingStrategy(MyTradingStrategy):
    def __init__(self, name= "My SMA Trading Stratefy", short_window=40, long_window=100):
        super().__init__(name)
        self.__short_window = short_window
        self.__long_window = long_window
    
    @property
    def short_window(self):
        return self.__short_window
    @property
    def long_window(self):
        return self.__long_window

    def generate_signals(self, price_data):
        if len(price_data) < self.long_window:
            return "Hold"
        
        short_ma = sum(price_data[-self.short_window:]) / self.short_window
        long_ma = sum(price_data[-self.long_window:]) / self.long_window
        
        if short_ma > long_ma:
            return "Buy"
        elif short_ma < long_ma:
            return "Sell"
        else:
            return "Hold"

mySMAObject = MySMATradingStrategy(long_window=5, short_window=3)
# print(mySMAObject.name)
# print(mySMAObject.short_window)
# print(mySMAObject.long_window)
# print(mySMAObject.generate_signals([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]))


Base Strategy
This method will be overridden by subclasses
Hold
My SMA Trading Stratefy
Buy
1000
2025-08-28 17:56:34.123235
Executing Buy amounting to 1000 using strategy My SMA Trading Stratefy at 2025-08-28 17:56:34.123235


In [25]:
class MyTrade:
    def __init__(self, strategy_name, signal, amount):   
        self.__strategy_name = strategy_name
        self.__signal = signal
        self.__amount = amount
        self.__timestamp = datetime.now()
        # self.__position = None
        # self.__trade_log = []

    @property
    def strategy_name(self):
        return self.__strategy_name
    @property
    def signal(self):
        return self.__signal
    @property
    def amount(self):
        return self.__amount
    @property
    def timestamp(self):
        return self.__timestamp

    # @property
    # def position(self):
    #     return self.__position

    # @property
    # def trade_log(self):
    #     return self.__trade_log
    def execute(self):
        print(f"Executing {self.__signal} amounting to {self.__amount} using strategy {self.__strategy_name} at {self.__timestamp}")
    # def execute_trade(self, price_data):
    #     signal = self.__strategy.generate_signals(price_data)
    #     if signal == "Buy" and self.__position != "Long":
    #         self.__position = "Long"
    #         self.__trade_log.append(("Buy", price_data[-1]))
    #         print(f"Executing Buy at {price_data[-1]}")
    #     elif signal == "Sell" and self.__position != "Short":
    #         self.__position = "Short"
    #         self.__trade_log.append(("Sell", price_data[-1]))
    #         print(f"Executing Sell at {price_data[-1]}")
    #     else:
    #         print("No action taken")
myTradeObject = MyTrade(strategy_name=mySMAObject.name ,signal=mySMAObject.generate_signals([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]), amount=1000)
print(myTradeObject.strategy_name)
print(myTradeObject.signal)
print(myTradeObject.amount)
print(myTradeObject.timestamp)
myTradeObject.execute()

My SMA Trading Stratefy
Buy
1000
2025-08-28 17:58:08.589791
Executing Buy amounting to 1000 using strategy My SMA Trading Stratefy at 2025-08-28 17:58:08.589791


In [26]:
class MockTradingAPI:
    def __init__(self,balance):
        self.__balance = balance
    
    @property
    def balance(self):
        return self.__balance
    
    def place_order(self, trade,price):
        if trade.signal == "Buy" and self.__balance >= trade.amount * price:
            self.__balance -= trade.amount * price
            print(f"Bought for {trade.amount} at price {price}. New balance: {self.__balance}")
        elif trade.signal == "Sell":
            self.__balance += trade.amount
            print(f"Sold for {trade.amount} at price {price}. New balance: {self.__balance}")
        else :
            print("No action taken or insufficient balance")
    
myAPIObject = MockTradingAPI(balance=10000)
print(myAPIObject.balance)
myAPIObject.place_order(myTradeObject, price=10)

10000
Bought for 1000 at price 10. New balance: 0


In [41]:
class MyTradingSystem:
    def __init__(self, strategy, api, symbol, initial_balance=10000):
        self.__strategy = strategy
        self.__api = api
        self.__initial_balance = initial_balance
        self.__symbol = symbol
        self.__price_data = []

    def fetch_price_data(self):
        data = yf.download(tickers=self.__symbol, period="1d", interval="1m")
        if not data.empty:
            self.price_data.append(data['Close'].iloc[-1]) 
            if len(self.price_data) > self.__strategy.long_window:
                self.price_data.pop(0)
            print(f"Fetched new price data: {self.price_data[-1]}")
        else:
            print("No data fetched")
    

    def run(self, new_price, trade_amount):
        self.fetch_price_data()
        signal = self.__strategy.generate_signals(self.__price_data)
        trade = MyTrade(strategy_name=self.__strategy.name, signal=signal, amount=trade_amount)
        trade.execute()
        self.__api.place_order(trade, new_price)
    
    @property
    def strategy(self):
        return self.__strategy
    @property
    def api(self):
        return self.__api
    @property
    def price_data(self):
        return self.__price_data
    @property
    def initial_balance(self):
        return self.__initial_balance
    @property
    def symbol(self):
        return self.__symbol
myTradingSystemObject = MyTradingSystem(strategy=mySMAObject, api=myAPIObject, symbol="AAPL", initial_balance=10000)
print(myTradingSystemObject.strategy.name)
print(myTradingSystemObject.api.balance)
print(myTradingSystemObject.price_data) # should be empty initially
print(myTradingSystemObject.initial_balance)
print(myTradingSystemObject.symbol)
myTradingSystemObject.run(new_price=150, trade_amount=1000) # Simulate a new price update
print(myTradingSystemObject.price_data)

/var/folders/zq/590d9tr54kd3q6bnnpjgyv0h0000gn/T/ipykernel_43732/1956264347.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=self.__symbol, period="1d", interval="1m")
[*********************100%***********************]  1 of 1 completed

My SMA Trading Stratefy
0
[]
10000
AAPL
Fetched new price data: Ticker
AAPL    230.440002
Name: 2025-08-27 19:59:00+00:00, dtype: float64
Executing Hold amounting to 1000 using strategy My SMA Trading Stratefy at 2025-08-28 18:37:32.217156
No action taken or insufficient balance
[Ticker
AAPL    230.440002
Name: 2025-08-27 19:59:00+00:00, dtype: float64]


In [37]:
data = yf.download(tickers="AAPL", period="1d", interval="1m")
print(data['Close'].iloc[-1]   )

/var/folders/zq/590d9tr54kd3q6bnnpjgyv0h0000gn/T/ipykernel_43732/3421438410.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers="AAPL", period="1d", interval="1m")
[*********************100%***********************]  1 of 1 completed

Ticker
AAPL    230.440002
Name: 2025-08-27 19:59:00+00:00, dtype: float64
